In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
import datetime

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [ ]:
# type: ignore
spark = SparkSession.builder.appName('Primeiro teste').getOrCreate()
path = os.path.join('C:\\Users', 'Hugo Martins',
                       'PycharmProjects', 'CODES', 'explotacion',
                       'penguins', 'data_clean.csv')
user_log = spark.read.csv(path)

In [18]:
user_log.take(5)


True

In [ ]:
user_log.printSchema()

In [ ]:
user_log.describe().show()

In [ ]:
user_log.describe('Column').show()

In [ ]:
user_log.count()

In [ ]:
user_log.select("column").dropDuplicates().sort("column").show()

In [ ]:
user_log.select(["userId", "firstname", "page", "song"])\
    .where(user_log.userId == "1046").collect()

In [24]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)

In [ ]:
user_log = user_log.withColumn("hour", get_hour(user_log.ts))

In [ ]:
songs_in_hour = user_log.filter(user_log.page == "NextSong") \
    .groupby(user_log.hour).count().orderBy(user_log.hour.cast("float"))

In [ ]:
songs_in_hour_pd = songs_in_hour.toPandas()
songs_in_hour_pd.hour = pd.to_numeric(songs_in_hour_pd.hour)

In [ ]:
plt.scatter(songs_in_hour_pd["hour"], songs_in_hour_pd["count"])
plt.xlim(-1, 24);
plt.ylim(0, 1.2 * max(songs_in_hour_pd["count"]))
plt.xlabel("Hour")
plt.ylabel("Songs played");

In [ ]:
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])

In [ ]:
user_log_valid.count()

In [ ]:
user_log.select("userId").dropDuplicates().sort("userId").show()

In [ ]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [ ]:
user_log_valid.count()

Primeiro se seleciona a coluna ID, dropa duplicata e ai sim conta os logs de usuário

In [ ]:
user_log.select('userID').dropDuplicates().where(user_log.gender == 'F').count()

In [ ]:
# filter for users with blank user id
blank_pages = user_log.filter(user_log.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = user_log.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

In [ ]:
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)